In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install -v python-mecab-ko

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [13]:
import pandas as pd
import numpy as np
import re
import urllib.request

In [14]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_data.csv")

('finance_data.csv', <http.client.HTTPMessage at 0x7fbcfdab11d0>)

In [15]:
data = pd.read_csv('finance_data.csv')
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
del data['sentence']

# 중복 제거
data.drop_duplicates(subset=['kor_sentence'], inplace=True)

# 한국어만 남기고 싶으면
data['kor_sentence'] = [re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',' ',s) for s in data['kor_sentence']]

data.columns = ['label', 'title']

In [16]:
news_labeling = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/labeling_news.csv')
news_labeling['title'] = [re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',' ',s) for s in news_labeling['title']]

In [17]:
data = pd.concat([data,news_labeling])

#인덱스 초기화 -> 0부터 정렬
data = data.reset_index(drop=True)

import mecab
mecab = mecab.MeCab()
data['tokenized'] = data['title'].apply(mecab.morphs)

# 불용어 제거
data['title']=None
stopwords = ['유로','라고','밝혔','대한','명','된다','부터','에게','등','리','관한','드','한다고','몇','코네','던','노','비','았','입니다','할','적','습니다','는다','니','저','로','것','핀란드','만','어','및','다고','고','된','될','약','게','며','지','합니다','아','또','때','겠','뿐','고','있','수','하','기','년','인','되','면','한다','에서','그','했','다','을','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for i in range(len(data)):
  stopwords_removed_sentence = [word for word in data.iloc[i,2] if not word in stopwords] # 불용어 제거
  data['title'][i] = stopwords_removed_sentence

del data['tokenized']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [18]:
senti_df = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/senti.csv')
del senti_df['Unnamed: 0']

In [19]:
data = pd.concat([data,senti_df])
data

,label,title
0,0,"[따르, 회사, 회사, 성장, 곳, 지만, 모든, 생산, 러시아, 옮길, 계획, 없]"
1,0,"[테크노폴리스, 컴퓨터, 기술, 통신, 분야, 일, 회사, 유치, 위해, 평방, 미..."
2,2,"[국제, 전자, 산업, 회사, 엘코, 텍, 탈린, 공장, 수십, 직원, 해고, 했으..."
3,1,"[새로운, 생산, 공장, 인해, 회사, 예상, 수요, 증가, 충족, 시킬, 능력, ..."
4,1,"[회사, 업데이트, 전략, 따르, 범위, 장기, 순, 매출, 성장, 목표, 으며, ..."
...,...,...
14806,0,반신반의
14807,1,신비
14808,2,아리송
14809,2,알쏭하


In [20]:
data.to_csv("concat_all.csv")